# Evaluate effectiveness of FilterFinder on all experiments  
Thomas Macrina  
May 17, 2017  

Inspecting meshsets for FilterFinder evaluation. 

Evaluating on gcloud with instance `alembic-davittest-across`  

We had success with our first batch of meshsets (`standard_224`), so now we're analyzing results from all the experiments here:  
`gs://image_assembly/experiments/davit_piritest/meshset_inspected/`  

In [1]:
using Alembic

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in anonymous at no file
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:320
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:320
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:320
 in require at ./loading.jl:259
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:320
 in require at ./loading.jl:259
 in include_string at loading.jl:282
 in execute_request at /home/ubuntu/.julia/v0.4/IJulia/src/execute_request.jl:164
 in eventloop at /home/ubuntu/.julia/v0.4/IJulia/src/IJulia.jl:138
 in anonymous at task.jl:447
while loading /home/ubuntu/.julia/v0.4/MKLSparse/src/./DSS/dss_generator.jl, in expression starting on line 3
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in anonymous at no file
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:320
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:320
 in inc

1,1002-1,1096_aligned.jls  
1,2002-1,2096_aligned.jls  
1,2-1,96_aligned.jls  
1,3002-1,3096_aligned.jls

0-thousands: original  
1-thousands: net trained on adj  
2-thousands: net trained on across  
3-thousands: bandpass

In [2]:
exp_dirs = ["standard_224", "standard_160", "across_224", "across_160"]
ms_filenames = Dict(
    "original" => "1,2-1,96_aligned.jls",
    "net_adj" => "1,1002-1,1096_aligned.jls",
    "net_across" => "1,2002-1,2096_aligned.jls",
    "bandpass" => "1,3002-1,3096_aligned.jls")

exp_dict = Dict()
for exp_dir in exp_dirs
    ms_dir = joinpath(homedir(), "davit", exp_dir)
    exp_dict[exp_dir] = [k => load(joinpath(ms_dir, v)) for (k,v) in ms_filenames]
end

Loading data from /home/ubuntu/davit/standard_224/1,3002-1,3096_aligned.jls
Loaded MeshSet from /home/ubuntu/davit/standard_224/1,3002-1,3096_aligned.jls
Loading data from /home/ubuntu/davit/standard_224/1,2-1,96_aligned.jls
Loaded MeshSet from /home/ubuntu/davit/standard_224/1,2-1,96_aligned.jls
Loading data from /home/ubuntu/davit/standard_224/1,1002-1,1096_aligned.jls
Loaded MeshSet from /home/ubuntu/davit/standard_224/1,1002-1,1096_aligned.jls
Loading data from /home/ubuntu/davit/standard_224/1,2002-1,2096_aligned.jls
Loaded MeshSet from /home/ubuntu/davit/standard_224/1,2002-1,2096_aligned.jls
Loading data from /home/ubuntu/davit/standard_160/1,3002-1,3096_aligned.jls
Loaded MeshSet from /home/ubuntu/davit/standard_160/1,3002-1,3096_aligned.jls
Loading data from /home/ubuntu/davit/standard_160/1,2-1,96_aligned.jls
Loaded MeshSet from /home/ubuntu/davit/standard_160/1,2-1,96_aligned.jls
Loading data from /home/ubuntu/davit/standard_160/1,1002-1,1096_aligned.jls
Loaded MeshSet from 

In [3]:
ms_names = collect(keys(ms_filenames))
comparisons = [("original", "net_adj"),
                ("original", "net_across"),
                ("original", "bandpass"),
                ("net_adj", "net_across"),
                ("net_adj", "bandpass"),
                ("net_across", "bandpass")];

### Compare meshsets  
A good kernel should create _more good matches_ and make it _easier to remove bad matches_.  

_More good matches_ means:  
1. a kernel finds good correspondences in locations where another kernel might not be able to find any.  
1. there is better coverage of matches (a more consistent density).   

_Easier to remove bad matches_ means:   
1. there are fewer bad matches overall.
1. we can filter out bad matches with an easily tuned filter (less need for manual intervention).  

To start evaluating these two criteria, we will inspect the f-score between pairs of kernels, as well as number of matches away from a perfect set of correspondences.

In [4]:
"""
Count TP, FP, FN between matchA & matchB (ground truth: matchA)
"""
function compare_matches(matchA, matchB)
    pA = Set(get_filtered_properties(matchA, "src_range"));
    pB = Set(get_filtered_properties(matchB, "src_range"));
    tp = length(intersect(pA, pB))
    fn = length(setdiff(pA, pB))
    fp = length(setdiff(pB, pA))
    return [tp, fn, fp]
end

"""
Compute F1-score
"""
function compute_f1(tp, fn, fp)
    return 2*tp / (2*tp + fn + fp)
end

"""
Compute F1-score from table of tp, fn, fp
"""
function compute_f1(pn)
    f1_array = zeros(size(pn,1))
    for i in 1:size(pn,1)
        f1_array[i] = compute_f1(pn[i,:]...)
    end
    f1 = compute_f1(sum(pn,1)[:]...)
    return f1, f1_array
end

compute_f1 (generic function with 2 methods)

In [5]:
"""
Count TP, FP, FN between all corresponding matches (ground truth: msA)
"""
function compare_meshsets(msA, msB)
    pn = zeros(Int64, length(msA.matches), 3)
    for (k, (matchA, matchB)) in enumerate(zip(msA.matches, msB.matches))
        pn[k,:] = [compare_matches(matchA, matchB)...]
    end
    return pn
end

compare_meshsets (generic function with 1 method)

In [6]:
function count_errors(ms, match)
    src_index = match.src_index
    mesh = get_mesh(ms, match.src_index)
    attempted_matches = length(mesh.src_nodes)
    possible_matches = length(get_properties(match, "src_range"))
    good_matches = length(get_filtered_properties(match, "src_range"))
    return [attempted_matches, possible_matches, good_matches]
end

count_errors (generic function with 1 method)

In [7]:
function list_discrepancies(matchA, matchB)
    pA = Set(get_filtered_properties(matchA, "src_range"));
    pB = Set(get_filtered_properties(matchB, "src_range"));
    fn = setdiff(pA, pB)
    fp = setdiff(pB, pA)
    return fn, fp
end

list_discrepancies (generic function with 1 method)

In [8]:
function compile_pn(ms_dict, comparisons)
    pn_results = Dict()
    for (nameA, nameB) in comparisons
        pn = compare_meshsets(ms_dict[nameA], ms_dict[nameB])
        pn_results[(nameA, nameB)] = pn
    end
    return pn_results
end

compile_pn (generic function with 1 method)

In [9]:
function compile_f1(pn_results)
    f1_results = Dict()
    for (name_pair, pn) in pn_results
        f1_results[name_pair] = compute_f1(pn)
    end
    return f1_results
end

compile_f1 (generic function with 1 method)

In [10]:
exp_pn = Dict()
for (exp, ms_dict) in exp_dict
    exp_pn[exp] = compile_pn(ms_dict, comparisons)
end

In [11]:
exp_f1 = Dict()
for (exp, pn_results) in exp_pn
    exp_f1[exp] = compile_f1(pn_results)
end

In [12]:
function compile_tables(pn_results, f1_results)
    pn_table = zeros(Int64, length(ms_names), length(ms_names), 3)
    f1_table = zeros(length(ms_names), length(ms_names))
    for (i, nameA) in enumerate(ms_names)
        for (j, nameB) in enumerate(ms_names)
            if (nameA, nameB) in keys(f1_results)
                pn_table[i,j,:] = sum(pn_results[(nameA, nameB)],1)
                f1_table[i,j] = f1_results[(nameA, nameB)][1]
            end
        end
    end
    return pn_table, f1_table
end

compile_tables (generic function with 1 method)

In [13]:
exp_tables = Dict()
for exp_name in keys(exp_pn)
    pn_results = exp_pn[exp_name]
    f1_results = exp_f1[exp_name]
    exp_tables[exp_name] = compile_tables(pn_results, f1_results)
end

In [18]:
s = collect([2,3,1])
println(ms_names[s])
println()
for (exp_name, (pn_table, f1_table)) in exp_tables
    println(exp_name)
    for (k, t) in enumerate(["tp", "fn", "fp"])
        println(t)
        println(pn_table[s,s,k])
    end
    println("f1")
    println(round(f1_table[s,s], 4))
end

ASCIIString["original","net_adj","bandpass"]

across_160
tp
[0 70178 69528
 0 0 70749
 0 0 0]
fn
[0 59 709
 0 0 1333
 0 0 0]
fp
[0 1904 1287
 0 0 66
 0 0 0]
f1
[0.0 0.9862 0.9858
 0.0 0.0 0.9902
 0.0 0.0 0.0]
across_224
tp
[0 71247 71170
 0 0 71972
 0 0 0]
fn
[0 6 83
 0 0 289
 0 0 0]
fp
[0 1014 814
 0 0 12
 0 0 0]
f1
[0.0 0.9929 0.9937
 0.0 0.0 0.9979
 0.0 0.0 0.0]
standard_160
tp
[0 142680 142681
 0 0 143954
 0 0 0]
fn
[0 76 75
 0 0 287
 0 0 0]
fp
[0 1561 1396
 0 0 123
 0 0 0]
f1
[0.0 0.9943 0.9949
 0.0 0.0 0.9986
 0.0 0.0 0.0]
standard_224
tp
[0 143672 143657
 0 0 144342
 0 0 0]
fn
[0 12 27
 0 0 94
 0 0 0]
fp
[0 764 702
 0 0 17
 0 0 0]
f1
[0.0 0.9973 0.9975
 0.0 0.0 0.9996
 0.0 0.0 0.0]


In [19]:
function compile_errors(ms_dict)
    error_results = Dict()
    for (name, ms) in ms_dict
        error_results[name] = zeros(Int64, 3)
        for match in ms.matches
            error_results[name] += count_errors(ms, match)
        end
    end
    return error_results
end

compile_errors (generic function with 1 method)

In [20]:
exp_error = Dict()
for (exp_name, ms_dict) in exp_dict
    exp_error[exp_name] = compile_errors(ms_dict)
end

In [24]:
for (exp_name, error_results) in exp_error
    println(exp_name)
    for (ms_name, error_array) in error_results
        println("$ms_name $error_array")
    end
    println()
end

across_160
bandpass [85399,72306,70815]
original [85399,72306,70237]
net_adj [85399,72347,72082]
net_across [81765,69251,68971]

across_224
bandpass [85399,72306,71984]
original [85399,72306,71253]
net_adj [85399,72347,72261]
net_across [81765,69251,69151]

standard_160
bandpass [170798,144500,144077]
original [170798,144500,142756]
net_adj [170798,144539,144241]
net_across [163530,138283,137809]

standard_224
bandpass [170798,144500,144359]
original [170798,144500,143684]
net_adj [170798,144539,144436]
net_across [163530,138302,138105]



In [27]:
for (exp_name, error_results) in exp_error
    println(exp_name)
    d = 144500 # hardcode at 144500, because the net kernel created areas that it could match
    if exp_name[1:6] == "across"
        d = 72306
    end
    for (ms_name, error_array) in error_results
        if ms_name == "net_across"
                println("$ms_name \t $(error_array[2] - error_array[3])")
        else
            println("$ms_name \t $(d - error_array[3])")
        end
    end
    println()
end

across_160
bandpass 	 1491
original 	 2069
net_adj 	 224
net_across 	 280

across_224
bandpass 	 322
original 	 1053
net_adj 	 45
net_across 	 100

standard_160
bandpass 	 423
original 	 1744
net_adj 	 259
net_across 	 474

standard_224
bandpass 	 141
original 	 816
net_adj 	 64
net_across 	 197



## PR curves   
Let's explore how robustly we can filter the matches generated by different kernels. This is a test of how easy we can remove matches.  

We'll vary parameters for the following filters:  
* distance  
* r delta
* r-max
* sigma (0.75?)  

We'll plot precision and recall curves for each filter as we vary the one parameter of the filter. We'll be comparing it to the ground truth we've created.  

We're writing out the important params as CSV files, because these Alembic gcloud instances don't plot easily.  

In [35]:
function compile_params(ms, name)
    total_correspondences = count_correspondences(ms)
    compiled_params = zeros(total_correspondences,5)
    n = 1
    for match in ms.matches
        m = count_correspondences(match)
        filtered = ones(m)
        filtered[collect(get_rejected_indices(match))] = 0
        dist = get_properties(match, "norm")
        r_max = get_properties(match, "r_max")
        sigma = get_properties(match, 0.75)
        r_delta = get_properties(match, 10)
        params = hcat(filtered, dist, r_max, sigma, r_delta)
        compiled_params[n:n+m-1,:] = params
        n += m
    end
    writedlm(joinpath(homedir(), "davit", string(name, "_params.csv")), compiled_params) 
end

compile_params (generic function with 1 method)

In [36]:
compile_params(exp_dict["standard_224"]["net_adj"], "standard_224_net_adj")

In [37]:
for (exp_name, ms_dict) in exp_dict
    for (ms_name, ms) in ms_dict
        fn = join([exp_name, ms_name], "_")
        compile_params(ms, fn)
    end
end